In [53]:
%load_ext autoreload
%autoreload 1

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [108]:
import json
import pandas as pd
import numpy as np
from collections import Counter
from pathlib import Path
import bz2
import re
import string
from pyparsing import Forward, Word, ZeroOrMore, alphanums, Suppress

In [55]:
%aimport config
from config import REPO

#### Read the data

In [39]:
file_test = "run-on-test.json"
with open(file_test) as f:
    test_data = json.load(f)

In [40]:
def sum_true(x):
    return sum([el[-1] for el in x])

In [55]:
frq = Counter([sum_true(el) for el in test_data])
for key, value in frq.most_common():
    print(f"Run-ons: {key}\nfrq: {value}, percentage: {value*100/len(test_data):.1f}%")

Run-ons: 1
frq: 145, percentage: 72.5%
Run-ons: 0
frq: 50, percentage: 25.0%
Run-ons: 2
frq: 5, percentage: 2.5%


In [60]:
def build_df(data):
    output = []
    for k,record in enumerate(data):
        temp = [{"id": k, "word": word, "label": label} for word, label in record]
        output.extend(temp)
    return pd.DataFrame(output)

In [221]:
test_df = build_df(test_data)
test_df.loc[test_df.id==79]

,id,label,word
1856,79,False,This
1857,79,False,is
1858,79,False,evidenced
1859,79,False,by
1860,79,False,results
1861,79,False,of
1862,79,False,a
1863,79,False,public
1864,79,False,polling
1865,79,True,exercise


In [77]:
# Check class imbalance
test_df.label.value_counts(normalize=True) * 100

False    96.700021
True      3.299979
Name: label, dtype: float64

#### Generate training data. Use Reddit data

In [63]:
folder = REPO / "cmv" / "all"
filename = "heldout_period_data.jsonlist.bz2"
with bz2.open(folder / filename, mode="rt") as f:
    lines = f.readlines()
    data = [json.loads(line) for line in lines]

In [79]:
def check_comment(comment):
    text = comment.get("body", "")
    if not text or text == "[deleted]":
        return False
    return True

In [137]:
def process_comment(comment):
    text = comment.get("body", "")
    text = re.sub(r"\(.*?\)", "", text)
    text = re.sub(r"[\n|\r]+", "", text)
    return text

In [145]:
comments = [process_comment(comment)+"\n" for el in data for comment in el["comments"] 
            if check_comment(comment)]
N = 10000
with open("data.txt", "w+") as f:
    f.writelines(comments[:N])